In [4]:
## change the following to where you have stored and/or extracted the files
path = '/Users/rhan39/Dropbox/Tool - Classes/ChBE 8803/Project/'

###################################################################

In [5]:
## load all packages used below
from PIL import Image
import glob
import numpy as np
import pandas as pd
import zipfile,io
import pylab as plt
import matplotlib.image as mpimg

In [6]:
## loads a user specified image (by index) and corresponding masks from ZIPPED stage1_train as np array, also displays png
 
def load_img(path,index,mask_index): #Load an image and masks
    #Lead it to the zip folder
    z=zipfile.ZipFile(path,'r') #Access zip folder
    names=z.namelist() #List of names
    name=names[index] #Get selected image
    name=name[0:-1] #Eliminate "/"
    mask_list=[]
    for string in names:
        if string.startswith(name+'/mask'):
            mask_list.append(string)
    mask_list=mask_list[1:-1] #list of masks
    mask_name=mask_list[mask_index] #Get individual mask
    img_raw=z.read('{}/images/{}.png'.format(name,name)) #Grab raw image
    mask_raw=z.read(mask_name) #Grab a mask
    img = io.BytesIO(img_raw) # convert image
    img = mpimg.imread(img) #numpy array
    img = np.flip(img,0) #flip image
    mask = io.BytesIO(mask_raw) #Convert mask
    mask = mpimg.imread(mask) #numpy array
    mask = np.flip(mask,0) #flip mask too
    
    return img, mask,mask_list

# WORKING EXAMPLE OF LOAD_IMG
bytes_io = load_img(path+'stage1_train.zip',0,0)
img = mpimg.imread(bytes_io) # numpy array
imgplot = plt.imshow(img,origin='lower')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/rhan39/Dropbox/Tool - Classes/ChBE 8803/Project/stage1_train.zip'

In [ ]:
## loads all images and masks from UNZIPPED stage1_train into both np array and dataframe

def png2idx(img_path):
    png = Image.open(img_path)
    h = png.size[0]
    w = png.size[1]
    np_png = np.array(png.getdata())
    df_png = pd.DataFrame(np_png)
    return h,w, np_png, df_png

def reindex(h, w, idx):
    row = idx // w
    col = idx % w
    new_idx = col*h+row
    return new_idx

train_list = glob.glob(path+'stage1_train/*') # ALL TRAINING SAMPLES
# train_list = [path+'stage1_train/0a7d30b252359a10fd298b638b90cb9ada3acced4e0c0e5a3692013f432ee4e9'] # 1st training sample, for testing purposes

img_mask_db = pd.DataFrame(columns=['image_ID','mask_ID']) # df containing ID for images and associated masks
for train in train_list:
    # images
    img_id = train.split('train/')[1]
    img_path = train+'/images/'+img_id+'.png'
    h,w, np_img, df_img = png2idx(img_path) # returns one-indexed image as np array AND pandas df
    # mask
    mask_list = glob.glob(train+'/masks/*.png')
    mask_id = []
    for mask in mask_list:
        m = mask.split('masks/')[1]
        mask_id = np.append(mask_id, m.split('.png'[0])[0]) # why does there need to be 2 [0]'s ??
        h,w, np_mask, df_mask = png2idx(mask) # returns one-indexed mask as np array AND pandas df
    img_mask_db = img_mask_db.append({'image_ID':img_id, 'mask_ID':mask_id}, ignore_index=True)